In [1]:
import pandas as pd
import plotly.graph_objects as go

In [2]:
#necesario para extraer imagenes
! pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importamos los resultados del rating de transparencia y el de Clarity

In [3]:
clarity=pd.read_csv('rating_clarity_ibex35.csv', sep=';')
rating_transparencia=pd.read_csv('resultados_rating_app_2.csv')

In [4]:
rating_transparencia=rating_transparencia.rename(columns={'empresas':'empresa'})

Generamos un dataframe diferente para los resultados de cada año

In [5]:
nuestro_2021=rating_transparencia[rating_transparencia.año==2021]
nuestro_2021=nuestro_2021[nuestro_2021.score!=0]
nuestro_2020=rating_transparencia[rating_transparencia.año==2020]
nuestro_2020=nuestro_2020[nuestro_2020.score!=0]
nuestro_2019=rating_transparencia[rating_transparencia.año==2019]
nuestro_2019=nuestro_2019[nuestro_2019.score!=0]
nuestro_2018=rating_transparencia[rating_transparencia.año==2018]
nuestro_2018=nuestro_2018[nuestro_2018.score!=0]
nuestro_2017=rating_transparencia[rating_transparencia.año==2017]
nuestro_2017=nuestro_2017[nuestro_2017.score!=0]

Generamos el dataframe con la comparacion de ambos ratings

In [6]:
comparacion=clarity.merge(rating_transparencia, on='empresa')
comparacion=comparacion[comparacion.año==2021]
comparacion=comparacion.loc[:,['empresa','rating_clarity','score','score_cualitativo']]

Se calculan los percentiles para los distintos rating y se observan en un grafico

In [7]:
import numpy as np
percentiles_clarity=[]
arry = comparacion['rating_clarity']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_clarity.append(percentile)


percentiles_nuestro=[]
arry = comparacion['score']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_nuestro.append(percentile)

percentiles_nuestro_cual=[]
arry = comparacion['score_cualitativo']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_nuestro_cual.append(percentile)

In [9]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_clarity,mode='lines', name='Rat. Clarity',line = dict(color = 'lightblue')))
fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_nuestro,mode='lines', name='Rat. Transparencia',line = dict(color = 'dodgerblue')))
fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_nuestro_cual,mode='lines', name='Rat. Transparencia Cual',line = dict(color = 'darkblue')))


fig.update_layout(

xaxis_title='Percentil',
yaxis_title="Score",
width=int(700),
yaxis_range=[0,105],
xaxis_range=[0,100],
plot_bgcolor='white'
    )
fig.update_xaxes(tickvals=[10,20,30,40,50,60,70,80,90,100],gridcolor='lightgrey')
fig.update_yaxes(tickvals=[10,20,30,40,50,60,70,80,90,100],gridcolor='lightgrey')

fig.update_layout(title_text='Comparación percentiles entre ratings ')
fig.show()
fig.write_image("comp_percentiles.png")

Gracias a este grafico podemos comparar dada una empresa cual es su score segun los diferentes ratings y en que percentil se encuentra para cada uno de ellos. Para ello se genera una funcion cuya entrada es la empresa a analizar.

In [10]:
def percentil_empresa(empresa):
  score_c=float(comparacion[comparacion.empresa==empresa].rating_clarity)
  score_n=float(comparacion[comparacion.empresa==empresa].score)
  score_nc=float(comparacion[comparacion.empresa==empresa].score_cualitativo)
  for i in range(0,99):
    izq=percentiles_clarity[i]
    derecho=percentiles_clarity[i+1]
    
    if score_c <= derecho and score_c>izq:
      percentil_c=i
  for i in range(0,99):
    izq=percentiles_nuestro[i]
    derecho=percentiles_nuestro[i+1]
    
    if score_n <= derecho and score_n>izq:
      percentil_n=i
  for i in range(0,99):
    izq=percentiles_nuestro_cual[i]
    derecho=percentiles_nuestro_cual[i+1]
    
    if score_nc <= derecho and score_nc>izq:
      percentil_nc=i
  titulo='Comparación entre rating - '+ empresa
  fig = go.Figure()



  fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_clarity,mode='lines', name='clarity',line = dict(color = 'lightblue')))
  fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_nuestro,mode='lines', name='nuestro',line = dict(color = 'dodgerblue')))
  fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_nuestro_cual,mode='lines', name='nuestro cual',line = dict(color = 'darkblue')))
 
  fig.add_trace(go.Scatter(x=[percentil_c,percentil_n,percentil_nc], y=[score_c,score_n,score_nc], mode='markers',name=empresa, marker=dict(size=[10,10,10],
                  color=[ 1,1,1])))

  fig.update_layout(

  xaxis_title='Percentil',
  yaxis_title="Score",
  width=int(700),
  yaxis_range=[0,105],
  xaxis_range=[0,100],
  plot_bgcolor='white'
      )
  fig.update_xaxes(tickvals=[10,20,30,40,50,60,70,80,90],gridcolor='lightgrey')
  fig.update_yaxes(tickvals=[10,20,30,40,50,60,70,80,90,100],gridcolor='lightgrey')
  fig.update_layout(transition_duration=500)
  fig.update_layout(title_text=titulo)
  fig.show()
  fig.write_image("comp_percentiles_empresa.png")

In [11]:
percentil_empresa('Iberdrola')

Debido a que tenemos el historico del rating de transparencia desde 2017 podemos analizar cual es la evolucion de los percentiles

In [12]:
import numpy as np
percentiles_2021=[]
arry = nuestro_2021['score']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_2021.append(percentile)

percentiles_2020=[]
arry = nuestro_2020['score']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_2020.append(percentile)

percentiles_2019=[]
arry = nuestro_2019['score']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_2019.append(percentile)

percentiles_2018=[]
arry = nuestro_2018['score']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_2018.append(percentile)

percentiles_2017=[]
arry = nuestro_2017['score']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_2017.append(percentile)

In [13]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_2021, name='2021',line = dict(color = 'salmon'),))
fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_2020, name='2020',line = dict(color = 'forestgreen'),))
fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_2019, name='2019',line = dict(color = 'turquoise'),))
fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_2018, name='2018',line = dict(color = 'gold'),))
fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_2017, name='2017',line = dict(color = 'mediumpurple'),))

fig.update_layout(

xaxis_title='Percentil',
yaxis_title="Score Rat. Transparencia",
width=int(700),
yaxis_range=[0,105],
xaxis_range=[0,100],
plot_bgcolor='white'
    )
fig.update_xaxes(tickvals=[10,20,30,40,50,60,70,80,90],gridcolor='lightgrey')
fig.update_yaxes(tickvals=[10,20,30,40,50,60,70,80,90,100],gridcolor='lightgrey')
fig.update_layout(transition_duration=500)
fig.update_layout(title_text='Comparación histórica de percentiles ')
fig.show()
fig.write_image("comp_percentiles_años.png")

In [14]:
import numpy as np
percentiles_2021=[]
arry = nuestro_2021['score_cualitativo']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_2021.append(percentile)

percentiles_2020=[]
arry = nuestro_2020['score_cualitativo']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_2020.append(percentile)

percentiles_2019=[]
arry = nuestro_2019['score_cualitativo']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_2019.append(percentile)

percentiles_2018=[]
arry = nuestro_2018['score_cualitativo']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_2018.append(percentile)

percentiles_2017=[]
arry = nuestro_2017['score_cualitativo']
for i in range(0,100,1):
  percentile = np.percentile(arry, i)
  percentiles_2017.append(percentile)

In [15]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_2021, name='2021',line = dict(color = 'salmon'),))
fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_2020, name='2020',line = dict(color = 'forestgreen'),))
fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_2019, name='2019',line = dict(color = 'turquoise'),))
fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_2018, name='2018',line = dict(color = 'gold'),))
fig.add_trace(go.Scatter(x=list(range(0,100)), y=percentiles_2017, name='2017',line = dict(color = 'mediumpurple'),))

fig.update_layout(

xaxis_title='Percentil',
yaxis_title="Score Rat. Transparencia Cualitativo",
width=int(700),
yaxis_range=[0,105],
xaxis_range=[0,100],
plot_bgcolor='white'
    )
fig.update_xaxes(tickvals=[10,20,30,40,50,60,70,80,90],gridcolor='lightgrey')
fig.update_yaxes(tickvals=[10,20,30,40,50,60,70,80,90,100],gridcolor='lightgrey')
fig.update_layout(transition_duration=500)
fig.update_layout(title_text='Comparación histórica de percentiles ')
fig.show()
fig.write_image("comp_percentiles_años_cual.png")